# Lectures 13-15

- extracts from lectures 13-15
- 16 will be covered in next course
- good to have some extra tools in your toolkit
- connecting duckdb to python

In [10]:
with open("sql/ingestion.sql", 'r') as sql_script:
    sql_code = sql_script.read()

print(sql_code)

CREATE SCHEMA IF NOT EXISTS staging;

CREATE TABLE IF NOT EXISTS staging.students AS (
    SELECT * FROM read_csv_auto('data/students.csv')
);

CREATE TABLE IF NOT EXISTS staging.teachers AS (
    SELECT * FROM read_csv_auto('data/teachers.csv')
);

CREATE TABLE IF NOT EXISTS staging.housing_sales AS (
    SELECT * FROM read_csv_auto('data/housing_sales.csv')
);

CREATE TABLE IF NOT EXISTS staging.neighborhoods AS (
    SELECT * FROM read_csv_auto('data/neighborhoods.csv')
);

CREATE TABLE IF NOT EXISTS staging.categories AS (
    SELECT * FROM read_csv_auto('data/categories.csv')
);

CREATE TABLE IF NOT EXISTS staging.sales AS (
    SELECT * FROM read_csv_auto('data/sales.csv')
);


## with statement to connect to a duckdb

- with handles connection and closes it down properly
- if this database doesn't exist -> it creates it for us

In [11]:
import duckdb

with duckdb.connect("data/lecture_13_15.duckdb") as conn:
    conn.execute(sql_code)
    describe = conn.execute("DESC;").df()

describe

,database,schema,name,column_names,column_types,temporary
0,lecture_13_15,staging,categories,"[category_id, category_name, parent_category_id]","[BIGINT, VARCHAR, BIGINT]",False
1,lecture_13_15,staging,housing_sales,"[sale_id, neighborhood_id, address, date_of_sa...","[VARCHAR, BIGINT, VARCHAR, DATE, VARCHAR, BIGI...",False
2,lecture_13_15,staging,neighborhoods,"[neighborhood_id, name, city, postal_code, med...","[BIGINT, VARCHAR, VARCHAR, BIGINT, BIGINT, BIG...",False
3,lecture_13_15,staging,sales,"[product_id, product_name, category_id, quanti...","[VARCHAR, VARCHAR, BIGINT, BIGINT, BIGINT]",False
4,lecture_13_15,staging,students,"[student_id, grade_math, class_name]","[VARCHAR, BIGINT, VARCHAR]",False
5,lecture_13_15,staging,teachers,"[class_name, teacher_name]","[VARCHAR, VARCHAR]",False


In [12]:
# this doesn't work as the connection is close 
# duckdb.sql("FROM staging.students;")

In [13]:
import pandas as pd
from utils import query_database

query_database("FROM staging.students;").head(3)

,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S003,78,B


## Lecture 13 - Subquery

- a query inside of another query
- gets a temporary result set, which can be used

In [ ]:
query_database(
    """--sql
               SELECT
                    AVG(grade_math)
               FROM staging.students
               ;
            """)

,avg(grade_math)
0,84.2


find all rows where score is higher than the average
- subquery inside paranthesis - evaluted first

In [17]:
query_database("""--sql
                SELECT
                    *
                FROM
                    staging.students
               WHERE grade_math > (
               SELECT
                    AVG(grade_math)
               FROM staging.students
               );
               
               """)

,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S004,88,B
3,S005,95,C
4,S009,89,E
5,S010,93,E


solve without subquery

In [ ]:
query_database("""--sql
                SELECT
                    *
                FROM
                    staging.students
               WHERE grade_math > 84.2
               """)

,student_id,grade_math,class_name
0,S001,92,A
1,S002,85,A
2,S004,88,B
3,S005,95,C
4,S009,89,E
5,S010,93,E


## Lecture 14 - Views
- a saved SQL query as a virtual table
- the query is executed everytime it is accessed
- no data are stored

## Purposes of Views
- restrict accesses to underlying tables
- simplify complex queries (for examples, involving joins of many tables) for users
- customerize data for different users
- maintain stable interface between underlying tables and downstream consumers (for example, when column names change in the underlying tables)

In [20]:
query_database("""--sql
    
    FROM staging.housing_sales;"""
).head()

,sale_id,neighborhood_id,address,date_of_sale,property_type,rooms,living_area_sqm,year_built,sale_price_usd
0,S-24001,1,124 Maple Crest Ave,2024-10-09,Apartment,2,58,2018,245575
1,S-24002,1,9 Birch Hollow Ln,2025-02-21,Townhouse,4,112,2006,454575
2,S-24003,2,77 Riverwalk Way,2025-03-18,Detached,5,168,1999,757625
3,S-24004,2,18 Marina Bluff Ct,2025-07-05,Apartment,3,86,2015,412775
4,S-24005,3,401 Quarry Gate Rd,2024-12-12,Semi-Detached,4,124,1987,381425


pick out average of property types and put into a view to give to downstream users

- if storing this as a table and the underlying data changes e.g. more houses are added then the "averages sales table" is wrong
- with a view - it will run the sql query when the downstream user is querying the view - which will give fresh results all the time

In [ ]:
query_database("""--sql
CREATE OR REPLACE VIEW staging.average_price_type AS
SELECT
    property_type,
    AVG(sale_price_usd)::INT as avg_price
FROM staging.housing_sales
GROUP BY property_type;
""")

,property_type,avg_price
0,Loft,316112
1,Townhouse,477913
2,Condo,312803
3,Detached,699105
4,Terraced,350075
5,Apartment,290361
6,Semi-Detached,391875
7,Bungalow,423225
